In [1]:
import numpy as np
import dask.array as da
from dask.distributed import Client, LocalCluster

In [2]:
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 15.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:53148,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 12
Started: Just now,Total memory: 15.64 GiB
Comm: tcp://127.0.0.1:53177,Total threads: 3
Dashboard: http://127.0.0.1:53178/status,Memory: 3.91 GiB
Nanny: tcp://127.0.0.1:53151,


In [3]:
# Create a large array and calculate the mean
x = da.random.random((10000, 10000), chunks=(1000, 1000))
y = x.mean()
y.compute()


0.5000015487642705

In [4]:
ones = da.ones((1000, 1000), chunks=(100, 100))
ones

dask.array<ones_like, shape=(1000, 1000), dtype=float64, chunksize=(100, 100), chunktype=numpy.ndarray>

In [5]:
sum_ones = ones.sum()
# sum_ones.visualize(filename="sum_ones.png")
sum_ones.compute()

1000000.0

In [6]:
# Load xarray dataset
import xarray as xr
fullpath = r"C:\Users\baptiste.menetrier\Desktop\devPy\phd\localisation\verlinden\verlinden_process_output\testcase1_0\ship\not_on_grid\intercorr0\output_intercorr0_noiseless.nc"
ds = xr.open_dataset(fullpath)

In [7]:
# Lets say we want to iterate over the ship time dimension and calculate the ambiguity surface for each time step
# We can use dask to parallelize the computation

# We want to use delayed to parallelize the computation
from dask import delayed
from misc import mult_along_axis


# Define the function that will be applied to each time step
def calculate_ambiguity_surface(ds, time, i_ship, ambiguity_surface):
    i_pair = 0
    lib_data = ds.library_corr.sel(idx_rcv_pairs=i_pair)
    event_data = ds.event_corr.sel(idx_rcv_pairs=i_pair)
    event_vector = event_data.sel(src_trajectory_time=time)

    amb_surf = mult_along_axis(
        lib_data,
        event_vector,
        axis=2,
    )
    autocorr_lib_0 = np.sum(lib_data.values**2, axis=2)
    autocorr_event_0 = np.sum(event_vector.values**2)
    # del lib_data, event_vector

    norm = np.sqrt(autocorr_lib_0 * autocorr_event_0)
    amb_surf = np.sum(amb_surf, axis=2) / norm  # Values in [-1, 1]
    amb_surf = (amb_surf + 1) / 2  # Values in [0, 1]
    ambiguity_surface[i_pair, i_ship, ...] = amb_surf

    return ambiguity_surface

ambiguity_surface_dim = ["idx_rcv_pairs", "src_trajectory_time", "lat", "lon"]
ambiguity_surface = np.empty(tuple(ds.dims[d] for d in ambiguity_surface_dim))

In [8]:
# Usual loop
for i_ship, time in enumerate(ds["src_trajectory_time"]):
    ambiguity_surface = calculate_ambiguity_surface(ds, time, i_ship, ambiguity_surface)

KeyboardInterrupt: 

In [80]:
client = Client(LocalCluster(n_workers=8))
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:54195/status,
Dashboard: http://127.0.0.1:54195/status,Workers: 8
Total threads: 16,Total memory: 15.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54196,Workers: 8
Dashboard: http://127.0.0.1:54195/status,Total threads: 16
Started: Just now,Total memory: 15.64 GiB
Comm: tcp://127.0.0.1:54244,Total threads: 2
Dashboard: http://127.0.0.1:54248/status,Memory: 1.96 GiB
Nanny: tcp://127.0.0.1:54199,


In [ ]:
# Using dask delayed
# Create a list of delayed objects
delayed_list = []
for i_ship, time in enumerate(ds["src_trajectory_time"]):
    delayed_list.append(
        delayed(calculate_ambiguity_surface)(ds, time, i_ship, ambiguity_surface)
    )

In [ ]:
result = da.compute(*delayed_list)

More usefull example with long task 

In [ ]:
ds

<xarray.Dataset> Size: 862MB
Dimensions:              (idx_rcv: 2, lat: 81, lon: 81, idx_rcv_pairs: 1,
                          idx_rcv_in_pair: 2, library_signal_time: 8192,
                          library_corr_lags: 16383, event_signal_time: 8192,
                          src_trajectory_time: 10, event_corr_lags: 16383)
Coordinates:
  * lon                  (lon) float64 648B -5.118 -5.117 -5.115 ... -5.002 -5.0
  * lat                  (lat) float64 648B 52.69 52.69 52.69 ... 52.76 52.76
  * idx_rcv              (idx_rcv) int32 8B 0 1
  * idx_rcv_pairs        (idx_rcv_pairs) int32 4B 0
  * idx_rcv_in_pair      (idx_rcv_in_pair) int32 8B 0 1
  * library_signal_time  (library_signal_time) float32 33kB 0.0 0.0025 ... 20.48
  * library_corr_lags    (library_corr_lags) int32 66kB -8191 -8190 ... 8191
  * event_signal_time    (event_signal_time) float32 33kB 0.0 0.0025 ... 20.48
  * src_trajectory_time  (src_trajectory_time) float32 40B 0.0 9.719 ... 87.47
  * event_corr_lags      (event_corr_lags) int32 66kB -8191 -8190 ... 8190 8191
Data variables: (12/16)
    lon_rcv              (idx_rcv) float64 16B ...
    lat_rcv              (idx_rcv) float64 16B ...
    r_from_rcv           (idx_rcv, lat, lon) float64 105kB ...
    delay_rcv            (idx_rcv, lat, lon) float64 105kB ...
    rcv_pairs            (idx_rcv_pairs, idx_rcv_in_pair) int32 8B ...
    rcv_signal_library   (idx_rcv, lat, lon, library_signal_time) float32 430MB ...
    ...                   ...
    az_src_rcv           (idx_rcv, src_trajectory_time) float32 80B ...
    rcv_signal_event     (idx_rcv, src_trajectory_time, event_signal_time) float32 655kB ...
    event_corr           (idx_rcv_pairs, src_trajectory_time, event_corr_lags) float32 655kB ...
    ambiguity_surface    (idx_rcv_pairs, src_trajectory_time, lat, lon) float64 525kB ...
    detected_pos_lon     (idx_rcv_pairs, src_trajectory_time) float64 80B ...
    detected_pos_lat     (idx_rcv_pairs, src_trajectory_time) float64 80B ...
Attributes:
    title:               Verlinden simulation with simple environment
    dx:                  100
    dy:                  100
    fullpath_populated:  C:\Users\baptiste.menetrier\Desktop\devPy\phd\locali...
    snr_dB:              Noiseless
    source_positions:    Not interpolated on grid
    src_pos:             not_on_grid
    long_name:           Ambiguity surface
    units:               dB
    detection_metric:    intercorr0
    fullpath_output:     C:\Users\baptiste.menetrier\Desktop\devPy\phd\locali...
    fullpath_analysis:   C:\Users\baptiste.menetrier\Desktop\devPy\phd\img\lo...

In [ ]:
def task(sig_lib):
    return sig_lib.mean()

In [ ]:
ds["lon"]
ds.dims["lon"]

81

In [10]:
ds.rcv_signal_library

<xarray.DataArray 'rcv_signal_library' (idx_rcv: 2, lat: 81, lon: 81,
                                        library_signal_time: 8192)> Size: 430MB
[107495424 values with dtype=float32]
Coordinates:
  * lon                  (lon) float64 648B -5.118 -5.117 -5.115 ... -5.002 -5.0
  * lat                  (lat) float64 648B 52.69 52.69 52.69 ... 52.76 52.76
  * idx_rcv              (idx_rcv) int32 8B 0 1
  * library_signal_time  (library_signal_time) float32 33kB 0.0 0.0025 ... 20.48
Attributes:
    long_name:  $s_{i}$

In [11]:
sig_lib = ds.rcv_signal_library.sel(idx_rcv=0).values
sig_lib

array([[[-3.44220680e-10, -1.13488031e-10,  1.74193437e-10, ...,
          1.17088431e-11, -2.59398530e-10, -4.02332279e-10],
        [-2.83288476e-10, -5.81982136e-11,  1.95583022e-10, ...,
         -3.53785543e-11, -2.67111971e-10, -3.66710995e-10],
        [-2.18098040e-10,  2.18194907e-11,  2.49811699e-10, ...,
         -1.11096014e-10, -3.06204090e-10, -3.49308971e-10],
        ...,
        [-9.64699765e-10, -4.61312266e-10,  2.75069051e-10, ...,
          2.25746435e-10, -5.04036590e-10, -9.81450810e-10],
        [-1.02809328e-09, -8.33537350e-10, -2.18189647e-10, ...,
          6.62275068e-10, -2.68247750e-11, -7.04433456e-10],
        [-8.94418151e-10, -9.86757120e-10, -5.81175663e-10, ...,
          8.98523089e-10,  3.66898956e-10, -3.51649099e-10]],

       [[-1.70300940e-10,  1.62474562e-10,  4.13978463e-10, ...,
         -2.63642136e-10, -4.53916904e-10, -4.16946505e-10],
        [-1.14181706e-10,  1.76321902e-10,  3.78098858e-10, ...,
         -2.61927480e-10, -4.06109119e

In [73]:
sig_lib_da = da.from_array(sig_lib)
sig_lib_da

dask.array<array, shape=(81, 81, 8192), dtype=float32, chunksize=(81, 81, 5114), chunktype=numpy.ndarray>

In [52]:
%%time 
# Usual way to calculate the mean
mean_sig_lib = sig_lib.mean(axis=2)


CPU times: total: 15.6 ms
Wall time: 24 ms


In [14]:
mean_sig_lib

array([[-9.5636830e-15, -9.4521000e-15, -9.2907893e-15, ...,
        -1.8886931e-14, -1.5197214e-14, -1.1575891e-14],
       [-9.6215828e-15, -9.7177176e-15, -9.4150118e-15, ...,
        -2.2994796e-14, -2.2505564e-14, -1.9402204e-14],
       [-9.4549985e-15, -9.6374392e-15, -9.6969856e-15, ...,
        -2.8771636e-14, -2.6284876e-14, -2.3704014e-14],
       ...,
       [ 1.1609230e-15,  1.1564507e-15,  1.1018069e-15, ...,
         2.0966268e-15,  2.2811207e-15,  2.5026232e-15],
       [ 1.1699626e-15,  1.1509484e-15,  1.1703556e-15, ...,
         1.6924938e-15,  1.9223582e-15,  2.0991780e-15],
       [ 1.1662288e-15,  1.1661204e-15,  1.1727950e-15, ...,
         1.0732450e-15,  1.2581726e-15,  1.4630326e-15]], dtype=float32)

Compute mean with dask 

In [68]:
%%time
means = sig_lib_da.mean(axis=2)

CPU times: total: 0 ns
Wall time: 2.63 ms


In [69]:
%%time 
means.compute()

CPU times: total: 938 ms
Wall time: 968 ms


array([[-9.58807754e-15, -9.47591682e-15, -9.26712828e-15, ...,
        -1.89128769e-14, -1.51793319e-14, -1.15878715e-14],
       [-9.63846245e-15, -9.69710084e-15, -9.40471185e-15, ...,
        -2.29502086e-14, -2.24659226e-14, -1.94439598e-14],
       [-9.46639278e-15, -9.61638538e-15, -9.65053775e-15, ...,
        -2.87734449e-14, -2.62456208e-14, -2.37361400e-14],
       ...,
       [ 1.16960342e-15,  1.14559851e-15,  1.10588960e-15, ...,
         2.08186468e-15,  2.30308089e-15,  2.48715301e-15],
       [ 1.17256804e-15,  1.15109745e-15,  1.16916635e-15, ...,
         1.71972760e-15,  1.92057945e-15,  2.10851234e-15],
       [ 1.16387748e-15,  1.17055210e-15,  1.18168550e-15, ...,
         1.07029050e-15,  1.25503855e-15,  1.46061006e-15]], dtype=float32)

Same thing with persist()

In [74]:
%%time
means = sig_lib_da.mean(axis=2).persist()

CPU times: total: 328 ms
Wall time: 337 ms


In [76]:
%%time 
means.compute()

CPU times: total: 31.2 ms
Wall time: 12.1 ms


array([[-9.5424428e-15, -9.4733317e-15, -9.2911146e-15, ...,
        -1.8885338e-14, -1.5203943e-14, -1.1587194e-14],
       [-9.6349252e-15, -9.7133469e-15, -9.4015541e-15, ...,
        -2.2980851e-14, -2.2503212e-14, -1.9436052e-14],
       [-9.4725253e-15, -9.6129396e-15, -9.6862012e-15, ...,
        -2.8778588e-14, -2.6270205e-14, -2.3760365e-14],
       ...,
       [ 1.1835964e-15,  1.1676993e-15,  1.1153730e-15, ...,
         2.0869537e-15,  2.2893505e-15,  2.5043444e-15],
       [ 1.1673029e-15,  1.1779518e-15,  1.1692714e-15, ...,
         1.6960446e-15,  1.9060003e-15,  2.1259849e-15],
       [ 1.1705995e-15,  1.1637691e-15,  1.1728899e-15, ...,
         1.0599703e-15,  1.2612185e-15,  1.4511368e-15]], dtype=float32)

In [85]:
x = da.random.random((10000, 10000, 10))
y = da.random.random((10000, 10000, 10))
z = (da.arcsin(x) + da.arccos(y)).sum(axis=(1, 2))
# z.persist()
z.compute()

array([156896.34558617, 156984.94568058, 157104.89580252, ...,
       156812.32755309, 157217.36634491, 157155.84265122])

In [81]:
%%time
x = da.random.random(10_000_000, chunks=(10_000_000,))
x.sum().compute()

CPU times: total: 312 ms
Wall time: 741 ms


5000393.748583506